# Yelp dataset Exploration
This notebook will explore the previously downloaded **Yelp** dataset available [here](https://www.yelp.com/dataset).<br>
The idea is to find a subset of the dataset that will work for analysis, than save that dataset as a separate set that is easily workable within the jupyter notebook environment.

In [72]:
# import packages
import pandas as pd
import numpy as np
import os
import re

# surpress warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# set general data path
data_path="/Users/justinwilliams/Dropbox/data620/yelp_dataset"

In [14]:
# set data paths
data_business_path=os.path.join(data_path,'yelp_academic_dataset_business.json')
data_reviews_path=os.path.join(data_path,"yelp_academic_dataset_review.json")
data_user_path=os.path.join(data_path,"yelp_academic_dataset_user.json")

In [19]:
# import into pandas
df_business=pd.read_json(data_business_path, lines=True)
df_reviews=pd.read_json(data_reviews_path,lines=True)
df_user=pd.read_json(data_user_path,lines=True)

## Businesses
Let's first look at the `business` dataset to filter by `city`, `state` and `category`.

In [21]:
# column list
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [23]:
# what states are in dataset
df_business['state'].value_counts()

PA     34039
FL     26330
TN     12056
IN     11247
MO     10913
LA      9924
AZ      9912
NJ      8536
NV      7715
AB      5573
CA      5203
ID      4467
DE      2265
IL      2145
TX         4
CO         3
WA         2
HI         2
MA         2
NC         1
UT         1
MT         1
MI         1
SD         1
XMS        1
VI         1
VT         1
Name: state, dtype: int64

Let's look at `Philadelphia,PA`.

In [67]:
# filter for philly
df_business_philly=df_business[(df_business['state'] == 'PA') & \
                               (df_business['city'] == 'Philadelphia')]

Let's look at categories within Philadelphia.

In [69]:
df_business_philly['categories'].value_counts()

Restaurants, Pizza                                                                                                          154
Pizza, Restaurants                                                                                                          124
Restaurants, Chinese                                                                                                         89
Nail Salons, Beauty & Spas                                                                                                   86
Chinese, Restaurants                                                                                                         84
                                                                                                                           ... 
Food, Japanese, Bubble Tea, Poke, Sushi Bars, Ramen, Restaurants                                                              1
Food, Sandwiches, Fast Food, Restaurants, Diners, Seafood, Street Vendors                               

Looks like `Restaurants, Pizza` is the highest, followed by `Pizza, Restaurant`.<br>
Let's use some regex to get anything that has `Restaurants` in the `categories` column.

In [60]:
del df_business_philly

In [78]:
# custom function
def has_restaurant(string):
    """
    Returns bool if match
    """
    if string and re.match('.*Restaurants.*', string):
        return True
    else:
        return False
    
# apply function to categories, create bool col
df_business_philly['has_restaurant'] = df_business_philly['categories'].apply(has_restaurant)

# filter
df_business_philly_rest=df_business_philly.loc[df_business_philly['has_restaurant']]\
    .reset_index(drop=True)


In [80]:
# preview
df_business_philly_rest.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,has_restaurant
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",True
1,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-...",True
2,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",True
3,QdN72BWoyFypdGJhhI5r7g,Bar One,767 S 9th St,Philadelphia,PA,19147,39.939825,-75.157447,4.0,65,0,"{'Smoking': 'u'no'', 'NoiseLevel': 'u'average'...","Cocktail Bars, Bars, Italian, Nightlife, Resta...","{'Monday': '16:0-0:0', 'Tuesday': '16:0-0:0', ...",True
4,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,4105 Main St,Philadelphia,PA,19127,40.022466,-75.218314,3.0,41,0,"{'RestaurantsReservations': 'False', 'Caters':...","Pizza, Restaurants, Salad, Soup","{'Tuesday': '17:0-21:30', 'Wednesday': '17:0-1...",True


Ok, now we have all the rows with that have `Restuarants` in the `categories` column.<br>
Let's filter the other datasets by the `business_id` column.

## Reviews
Let's take a look at the reviews dataset.

In [92]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


In [82]:
# columns
df_reviews.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

Ok, there is a `business_id` column, we need to filter all the rows in `df_reviews` by the business id from `df_business_philly_rest`

In [85]:
# merge df and create indicator column
merged_df = df_business_philly_rest.merge(df_reviews, on='business_id', 
                                          how='left', indicator=True)


Filter by columns in `df_review`.

In [99]:
df_reviews_philly_rest=merged_df[merged_df.columns[merged_df.columns.isin(df_reviews.columns)]]

In [107]:
df_reviews_philly_rest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687289 entries, 0 to 687288
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   business_id  687289 non-null  object        
 1   review_id    687289 non-null  object        
 2   user_id      687289 non-null  object        
 3   useful       687289 non-null  int64         
 4   funny        687289 non-null  int64         
 5   cool         687289 non-null  int64         
 6   text         687289 non-null  object        
 7   date         687289 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 47.2+ MB


In [100]:
df_reviews_philly_rest.head()

,business_id,review_id,user_id,useful,funny,cool,text,date
0,MTSW4McQd7CbVtyjqoe9mw,BXQcBN0iAi1lAUxibGLFzA,6_SpY41LIHZuIaiDs5FMKA,0,0,1,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,MTSW4McQd7CbVtyjqoe9mw,uduvUCvi9w3T2bSGivCfXg,tCXElwhzekJEH6QJe3xs7Q,3,1,2,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,MTSW4McQd7CbVtyjqoe9mw,a0vwPOqDXXZuJkbBW2356g,WqfKtI-aGMmvbA9pPUxNQQ,0,0,0,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,MTSW4McQd7CbVtyjqoe9mw,MKNp_CdR2k2202-c8GN5Dw,3-1va0IQfK-9tUMzfHWfTA,5,0,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,MTSW4McQd7CbVtyjqoe9mw,D1GisLDPe84Rrk_R4X2brQ,EouCKoDfzaVG0klEgdDvCQ,2,1,1,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35


## Users
Let's look at the users dataset

In [101]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 333.7+ MB


Let's look at the columns.

In [83]:
df_user.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

Ok, we can filer on `user_id`, which won't necessarly give us ONLY reviews for restaurants in Phildelphia, but could give us additional info on the users.

In [103]:
# merge df and create indicator column
merged_df2 = df_reviews_philly_rest.merge(df_user, on='user_id', 
                                          how='left', indicator=True)

In [104]:
df_user_filtered=merged_df2[merged_df2.columns[merged_df2.columns.isin(df_user.columns)]]

In [105]:
df_user_filtered.head()

,user_id,name,review_count,yelping_since,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,6_SpY41LIHZuIaiDs5FMKA,K,38,2009-12-29 09:27:50,,w-BH_CEsxR1h7VBbtrxFIA,1,3.08,0,0,0,0,0,0,0,0,0,1,0
1,tCXElwhzekJEH6QJe3xs7Q,Rob,163,2008-04-09 02:40:37,"2008,2009,2013,2014","VAAIsM7fz68gZhvHHyu1Aw, BqBQGyo_kG7yRRrMkpl5rg...",23,3.60,15,5,6,1,0,11,28,46,46,26,20
2,WqfKtI-aGMmvbA9pPUxNQQ,Kassi,278,2013-03-27 05:46:52,,"7AAFD2TbTe5rqaFpG_WhTw, Txa_sNMUSHfNzHwaV034fQ...",6,3.72,0,1,0,0,0,3,8,2,2,5,0
3,3-1va0IQfK-9tUMzfHWfTA,Katherine,28,2011-12-26 17:28:28,,"lYWmSipq8RFhl_mjr_B_8g, tJWiWoh13ex8x3eblsns6A...",1,3.47,0,0,0,0,0,0,0,0,0,0,0
4,EouCKoDfzaVG0klEgdDvCQ,Linda,164,2011-08-31 21:39:28,"2012,2013,2014,2015","TxaGyXjxNYxtDr1PJFmwtA, 2cMLyjyiz97TkTVSRQ0Tlg...",24,3.78,17,3,0,1,4,24,51,35,35,13,5


In [108]:
df_user_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687289 entries, 0 to 687288
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             687289 non-null  object 
 1   name                687289 non-null  object 
 2   review_count        687289 non-null  int64  
 3   yelping_since       687289 non-null  object 
 4   elite               687289 non-null  object 
 5   friends             687289 non-null  object 
 6   fans                687289 non-null  int64  
 7   average_stars       687289 non-null  float64
 8   compliment_hot      687289 non-null  int64  
 9   compliment_more     687289 non-null  int64  
 10  compliment_profile  687289 non-null  int64  
 11  compliment_cute     687289 non-null  int64  
 12  compliment_list     687289 non-null  int64  
 13  compliment_note     687289 non-null  int64  
 14  compliment_plain    687289 non-null  int64  
 15  compliment_cool     687289 non-nul

## Pickle results
Ok, now that we have manageable filtered dataset:
- Philadelphia restaurant businesses
- Reviews of those restaurants
- Profiles of those reviewers

We can pickle the df and analyze further in another notebook.

In [109]:
# pickle df
df_business_philly_rest.to_pickle("./data/df_business_philly_rest.pkl")
df_reviews_philly_rest.to_pickle("./data/df_revews_philly_rest.pkl")
df_user_filtered.to_pickle("./data/df_user_filtered.pkl")